# Shap-E

Shap-E는 비디오 게임 개발, 인테리어 디자인 및 건축에 사용할 수 있는 3D 에셋 생성을 위한 조건부 모델입니다. 대규모 3D 에셋 데이터 세트에서 훈련되었으며 각 객체의 더 많은 뷰를 렌더링하고 4K 대신 16K 포인트 클라우드를 생성하도록 후처리되었습니다. Shap-E 모델은 두 단계로 훈련됩니다.

1. 인코더는 3D 에셋의 포인트 클라우드와 렌더링된 뷰를 받아 에셋을 나타내는 암시적 함수의 매개변수를 출력합니다.
2. 확산 모델은 인코더에서 생성된 잠재 공간에서 훈련되어 신경 방사 필드(NeRF) 또는 텍스처가 있는 3D 메시를 생성하여 다운스트림 애플리케이션에서 3D 에셋을 더 쉽게 렌더링하고 사용할 수 있도록 합니다.

이 가이드에서는 Shap-E를 사용하여 자신만의 3D 에셋 생성을 시작하는 방법을 보여줍니다!

시작하기 전에 다음 라이브러리가 설치되어 있는지 확인하십시오.

In [ ]:
# Colab에서 필요한 라이브러리를 설치하려면 주석 처리를 제거하십시오.
#!pip install -q diffusers transformers accelerate trimesh

## 텍스트-3D

3D 객체의 GIF를 생성하려면 [ShapEPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/shap_e#diffusers.ShapEPipeline)에 텍스트 프롬프트를 전달합니다. 파이프라인은 3D 객체를 만드는 데 사용되는 이미지 프레임 목록을 생성합니다.

In [ ]:
import torch
from diffusers import ShapEPipeline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pipe = ShapEPipeline.from_pretrained("openai/shap-e", torch_dtype=torch.float16, variant="fp16")
pipe = pipe.to(device)

guidance_scale = 15.0
prompt = ["폭죽", "생일 컵케이크"]

images = pipe(
    prompt,
    guidance_scale=guidance_scale,
    num_inference_steps=64,
    frame_size=256,
).images

이제 [export_to_gif()](https://huggingface.co/docs/diffusers/main/en/api/utilities#diffusers.utils.export_to_gif) 함수를 사용해 이미지 프레임 리스트를 3D 오브젝트의 gif로 변환합니다.

In [ ]:
from diffusers.utils import export_to_gif

export_to_gif(images[0], "firecracker_3d.gif")
export_to_gif(images[1], "cake_3d.gif")

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/shap_e/firecracker_out.gif"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">프롬프트 = "폭죽"</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/shap_e/cake_out.gif"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">프롬프트 = "생일 컵케이크"</figcaption>
  </div>
</div>

## 이미지-3D

다른 이미지에서 3D 객체를 생성하려면 [ShapEImg2ImgPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/shap_e#diffusers.ShapEImg2ImgPipeline)을 사용합니다. 기존 이미지를 사용하거나 완전히 새로운 이미지를 생성할 수 있습니다. [Kandinsky 2.1](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../api/pipelines/kandinsky) 모델을 사용하여 새 이미지를 생성해 보겠습니다.

In [ ]:
from diffusers import DiffusionPipeline
import torch

prior_pipeline = DiffusionPipeline.from_pretrained("kandinsky-community/kandinsky-2-1-prior", torch_dtype=torch.float16, use_safetensors=True).to("cuda")
pipeline = DiffusionPipeline.from_pretrained("kandinsky-community/kandinsky-2-1", torch_dtype=torch.float16, use_safetensors=True).to("cuda")

prompt = "치즈버거, 흰색 배경"

image_embeds, negative_image_embeds = prior_pipeline(prompt, guidance_scale=1.0).to_tuple()
image = pipeline(
    prompt,
    image_embeds=image_embeds,
    negative_image_embeds=negative_image_embeds,
).images[0]

image.save("burger.png")

치즈버거를 [ShapEImg2ImgPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/shap_e#diffusers.ShapEImg2ImgPipeline)에 전달하여 3D 표현을 생성합니다.

In [ ]:
from PIL import Image
from diffusers import ShapEImg2ImgPipeline
from diffusers.utils import export_to_gif

pipe = ShapEImg2ImgPipeline.from_pretrained("openai/shap-e-img2img", torch_dtype=torch.float16, variant="fp16").to("cuda")

guidance_scale = 3.0
image = Image.open("burger.png").resize((256, 256))

images = pipe(
    image,
    guidance_scale=guidance_scale,
    num_inference_steps=64,
    frame_size=256,
).images

gif_path = export_to_gif(images[0], "burger_3d.gif")

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/shap_e/burger_in.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">치즈버거</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/shap_e/burger_out.gif"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">3D 치즈버거</figcaption>
  </div>
</div>

## 메시 생성

Shap-E는 다운스트림 애플리케이션을 위해 렌더링할 텍스처가 있는 메시 출력을 생성할 수도 있는 유연한 모델입니다. 이 예에서는 🤗 데이터 세트 라이브러리가 [데이터 세트 뷰어](https://huggingface.co/docs/hub/datasets-viewer#dataset-preview)에서 렌더링할 수 있는 `glb` 파일의 메시 시각화를 지원하므로 출력을 `glb` 파일로 변환합니다.

`output_type` 매개변수를 `"mesh"`로 지정하여 [ShapEPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/shap_e#diffusers.ShapEPipeline) 및 [ShapEImg2ImgPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/shap_e#diffusers.ShapEImg2ImgPipeline) 모두에 대한 메시 출력을 생성할 수 있습니다.

In [ ]:
import torch
from diffusers import ShapEPipeline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pipe = ShapEPipeline.from_pretrained("openai/shap-e", torch_dtype=torch.float16, variant="fp16")
pipe = pipe.to(device)

guidance_scale = 15.0
prompt = "생일 컵케이크"

images = pipe(prompt, guidance_scale=guidance_scale, num_inference_steps=64, frame_size=256, output_type="mesh").images

`export_to_ply()` 함수를 사용하여 메시 출력을 `ply` 파일로 저장합니다.

<Tip>

선택적으로 `export_to_obj()` 함수를 사용하여 메시 출력을 `obj` 파일로 저장할 수 있습니다. 다양한 형식으로 메시 출력을 저장하는 기능은 다운스트림 사용에 더 많은 유연성을 제공합니다!

</Tip>

In [ ]:
from diffusers.utils import export_to_ply

ply_path = export_to_ply(images[0], "3d_cake.ply")
print(f"폴더에 저장됨: {ply_path}")

그런 다음 trimesh 라이브러리를 사용하여 `ply` 파일을 `glb` 파일로 변환할 수 있습니다.

In [ ]:
import trimesh

mesh = trimesh.load("3d_cake.ply")
mesh_export = mesh.export("3d_cake.glb", file_type="glb")

기본적으로 메시 출력은 아래쪽 시점에서 초점을 맞추지만 회전 변환을 적용하여 기본 시점을 변경할 수 있습니다.

In [ ]:
import trimesh
import numpy as np

mesh = trimesh.load("3d_cake.ply")
rot = trimesh.transformations.rotation_matrix(-np.pi / 2, [1, 0, 0])
mesh = mesh.apply_transform(rot)
mesh_export = mesh.export("3d_cake.glb", file_type="glb")

데이터 세트 뷰어로 시각화하려면 메시 파일을 데이터 세트 리포지토리에 업로드하십시오!

<div class="flex justify-center">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/3D-cake.gif"/>
</div>